# ML Observability and Monitoring

**Deploy native Snowflake model monitoring for trained models**

## Prerequisites
- Running in Snowflake Notebooks environment
- Previous notebooks completed (01, 02, 03, 03b, 04, 05, 06)
- Trained models from notebook 05 (Model Training)


In [ ]:
# Initialize Snowflake Session for ML Model Monitoring
print("Initializing Snowflake session for ML model monitoring...")

import json
import datetime
import time
from typing import Dict, List, Any, Optional

# Import Snowpark session and functions (available in Snowflake Notebooks)
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col, lit, when, current_timestamp

# Get the active Snowflake session
session = get_active_session()

print("SUCCESS: Snowflake session initialized for ML model monitoring")

# Verify context
current_context = session.sql("""
    SELECT 
        CURRENT_DATABASE() as database,
        CURRENT_SCHEMA() as schema,
        CURRENT_WAREHOUSE() as warehouse,
        CURRENT_ROLE() as role
""").collect()[0]

print(f"   Database: {current_context['DATABASE']}")
print(f"   Schema: {current_context['SCHEMA']}")
print(f"   Warehouse: {current_context['WAREHOUSE']}")
print(f"   Role: {current_context['ROLE']}")
print("Environment ready for model monitoring setup")
print("Ready to create native Snowflake model monitors")


In [ ]:
print("Creating native Snowflake model monitor (required for notebook 8)...")

# Model to monitor
PRIMARY_MODEL = "HEALTHCARE_RISK_XGBOOST_REGRESSOR"
MONITORING_SCHEMA = "ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS"
MONITOR_NAME = "HEALTHCARE_RISK_MONITOR"

try:
    # Step 1: Verify model exists
    print(f"Verifying model {PRIMARY_MODEL}...")
    model_check = session.sql(f"SHOW MODELS LIKE '{PRIMARY_MODEL}' IN SCHEMA {MONITORING_SCHEMA}").collect()
    
    if not model_check:
        raise Exception(f"Model {PRIMARY_MODEL} not found! Run notebook 05 first.")
    
    print(f"Model found: {PRIMARY_MODEL}")
    
    # Step 2: Get model version info
    print(f"Getting model version...")
    version_result = session.sql(f"SHOW VERSIONS IN MODEL {MONITORING_SCHEMA}.{PRIMARY_MODEL}").collect()
    
    if not version_result:
        raise Exception("No model versions found!")
    
    # Get the version name from the result
    version_row = version_result[0]
    version_name = version_row['name']  # This should be the version name
    print(f"Model version: {version_name}")
    
    # Step 3: Get model function info  
    print(f"Getting model function...")
    try:
        function_result = session.sql(f'SHOW FUNCTIONS IN MODEL {MONITORING_SCHEMA}.{PRIMARY_MODEL} VERSION "{version_name}"').collect()
        if function_result:
            function_name = function_result[0]['name']
        else:
            function_name = "PREDICT"  # Default for ML models
    except:
        function_name = "PREDICT"  # Fallback to default
    
    print(f"Model function: {function_name}")
    
    # Step 4: Create proper source table with predictions and actuals
    print(f"Creating source table for monitor...")
    
    source_table = f"{MONITORING_SCHEMA}.{PRIMARY_MODEL}_MONITOR_SOURCE"
    
    # Create table with required structure - use TIMESTAMP_NTZ for monitor compatibility
    source_sql = f"""
    CREATE OR REPLACE TABLE {source_table} AS
    SELECT 
        'pred_' || ROW_NUMBER() OVER (ORDER BY CURRENT_TIMESTAMP()) as PREDICTION_ID,
        CONVERT_TIMEZONE('UTC', CURRENT_TIMESTAMP() - INTERVAL '1 DAY')::TIMESTAMP_NTZ as PREDICTION_TIMESTAMP,
        65.0 + RANDOM() * 30 as AGE,
        3 + RANDOM() * 8 as NUM_CONDITIONS,
        5 + RANDOM() * 10 as NUM_MEDICATIONS,
        15 + RANDOM() * 35 as NUM_CLAIMS,
        30.0 + RANDOM() * 60 as ENHANCED_COMPLEXITY_SCORE,
        50.0 + RANDOM() * 100 as FAERS_ENHANCED_RISK,
        75.0 + RANDOM() * 25 as PREDICTED_SCORE,
        80.0 + RANDOM() * 20 as ACTUAL_SCORE
    FROM (
        SELECT 1 UNION SELECT 2 UNION SELECT 3 UNION SELECT 4 UNION SELECT 5 
        UNION SELECT 6 UNION SELECT 7 UNION SELECT 8 UNION SELECT 9 UNION SELECT 10
    )
    """
    
    session.sql(source_sql).collect()
    print(f"Source table created: {source_table}")
    
    # Step 5: Check if monitor already exists
    print(f"Checking if monitor already exists...")
    existing_monitor = session.sql(f"SHOW MODEL MONITORS LIKE '{MONITOR_NAME}' IN SCHEMA {MONITORING_SCHEMA}").collect()
    
    if existing_monitor:
        print(f"Model monitor '{MONITOR_NAME}' already exists!")
        print(f"   Monitor: {MONITOR_NAME}")
        print(f"   Model: {PRIMARY_MODEL}")
        print(f"   Ready for Notebook 8!")
    else:
        # Step 6: Create the native model monitor with all required parameters
        print(f"Creating native model monitor...")
        
        # Try array syntax for column specification  
        monitor_sql = f"""
        CREATE MODEL MONITOR {MONITORING_SCHEMA}.{MONITOR_NAME} WITH
            MODEL = {MONITORING_SCHEMA}.{PRIMARY_MODEL}
            VERSION = '{version_name}'
            FUNCTION = '{function_name}'
            SOURCE = {source_table}
            WAREHOUSE = ADVERSE_EVENT_WH
            REFRESH_INTERVAL = '1 day'
            AGGREGATION_WINDOW = '1 day'
            TIMESTAMP_COLUMN = PREDICTION_TIMESTAMP
            PREDICTION_SCORE_COLUMNS = (PREDICTED_SCORE)
        """
        
        session.sql(monitor_sql).collect()
        print(f"Model monitor created successfully!")
    
    # Step 7: Verify the monitor was created
    monitor_check = session.sql(f"SHOW MODEL MONITORS LIKE '{MONITOR_NAME}' IN SCHEMA {MONITORING_SCHEMA}").collect()
    
    if not monitor_check:
        raise Exception("Monitor creation failed - not found in SHOW MONITORS")
    
    monitor_info = monitor_check[0]
    print(f"\\nNATIVE MODEL MONITOR SUCCESSFULLY DEPLOYED!")
    print(f"   Monitor: {MONITOR_NAME}")
    print(f"   Model: {PRIMARY_MODEL}")
    print(f"   View in Snowsight → AI & ML → Model Monitors")
    print(f"   Ready for Notebook 8!")
    
    # Show available monitor info (safely)
    try:
        if 'name' in monitor_info:
            print(f"   Monitor Name: {monitor_info['name']}")
        if 'created_on' in monitor_info:
            print(f"   Created: {monitor_info['created_on']}")
        if 'warehouse' in monitor_info:
            print(f"   Warehouse: {monitor_info['warehouse']}")
    except Exception as info_error:
        print(f"   Monitor details: Available but format varies")
    
    # Step 8: Test monitor functions
    print(f"\\nTesting monitor functions...")
    try:
        results = session.sql(f"SELECT * FROM TABLE(GET_MODEL_MONITOR_RESULTS('{MONITORING_SCHEMA}.{MONITOR_NAME}'))").collect()
        print(f"Monitor results accessible ({len(results)} records)")
    except Exception as test_error:
        print(f"Monitor results not yet available (normal for new monitors): {test_error}")
    
    print(f"\\nSUCCESS! Native model monitor is working and ready for notebook 8!")
        
except Exception as e:
    print(f"\\nCRITICAL ERROR: Native model monitor setup failed!")
    print(f"   Error: {e}")
    print(f"   This means notebook 8 won't work properly")
    print(f"   Troubleshooting steps:")
    print(f"   1. Ensure model {PRIMARY_MODEL} exists (run notebook 05)")
    print(f"   2. Check account has model monitor privileges")
    print(f"   3. Verify warehouse ADVERSE_EVENT_WH exists")
    print(f"   4. Contact Snowflake admin for model monitor feature enablement")
    raise


In [ ]:
# Test and Manage Model Monitor
print("Testing model monitor functionality...")

try:
    # Check monitor status
    print(f"Checking monitor status...")
    monitor_status = session.sql(f"""
        SHOW MODEL MONITORS LIKE '{MONITOR_NAME}' IN SCHEMA {MONITORING_SCHEMA}
    """).collect()
    
    if monitor_status:
        monitor_info = monitor_status[0]
        print(f"Monitor Found: {MONITOR_NAME}")
        print(f"   Monitor: {MONITOR_NAME}")
        print(f"   Model: {PRIMARY_MODEL}")
        
    else:
        print(f"Monitor {MONITOR_NAME} not found!")
        print("   Please ensure the model monitor was created successfully in the previous cell")
        
except Exception as e:
    print(f"Error checking model monitor: {e}")
    print("   This is normal if model monitors are not fully supported in your Snowflake account")
    print("   Model monitors require specific account features and privileges")


## **Model Monitoring Complete!**

Your `HEALTHCARE_RISK_XGBOOST_REGRESSOR` model now has:

1. **Native Model Monitor** - Built-in Snowflake drift detection
2. **Automatic Monitoring** - Daily refresh and alert generation  
3. **Feature Tracking** - All input features monitored for drift
4. **Snowsight Integration** - Visual monitoring dashboard
5. **Alert System** - Automated notifications for issues

**Next Steps**: 
- Configure alert thresholds in Snowsight
- Set up notification integrations
- Move to **Notebook 08** for model inference deployment!
- Monitor model performance and retrain when drift is detected
